# City Graph
Here we are going to look at a subset of the cities database.  We are going to look for "local" story connections rather than big news items.  more specifically we will look at cities that appear in at least 3 but no more than 5 stories in days news.    This leaves out big cities like washington DC.  Next we will build a graph that we can use to explore the stories and cities that share these stories.  

The final graph will be interact and allow us to select city nodes and the stories they appear in  and the links to the other cities in those stories.  

In [4]:
import csv

In [2]:
#!pip install pydocumentdb

In [3]:
import pydocumentdb;
import pydocumentdb.document_client as document_client

In [5]:
config = { 
    'ENDPOINT': 'https://bookdocdb.documents.azure.com',
    'MASTERKEY': 'VHpAe9o2INSzEx5MaPMu2Yy9aFOXe4yLNPGnzkr7oDe0DA6wFocezMjUIodZp1xyGZ6aFjZnhNNBEih8d6dPog==',
    'DOCUMENTDB_DATABASE': 'db3',
    'DOCUMENTDB_COLLECTION': 'cities'
};

In [6]:
# Initialize the Python DocumentDB client
client = document_client.DocumentClient(config['ENDPOINT'], {'masterKey': config['MASTERKEY']})

In [7]:
try:
    db = client.CreateDatabase({ 'id': config['DOCUMENTDB_DATABASE'] }, )
except:
    db_id = config['DOCUMENTDB_DATABASE']
    db_query = "select * from r where r.id = '{0}'".format(db_id)
    db = list(client.QueryDatabases(db_query))[0]

INFO:Starting new HTTPS connection (1): bookdocdb.documents.azure.com
INFO:Starting new HTTPS connection (1): bookdocdb-northcentralus.documents.azure.com


In [8]:
# Create collection options
options = {
    'offerEnableRUPerMinuteThroughput': True,
    'offerVersion': "V2",
    'offerThroughput': 400
}

In [9]:
try:
    collection = client.CreateCollection(db['_self'], { 'id': config['DOCUMENTDB_COLLECTION'] }, options)
except:
    collection = next((coll for coll in client.ReadCollections(db['_self']) if coll['id']==config['DOCUMENTDB_COLLECTION']))

extrace the cities for each url in the database

In [10]:
db_query = {'query':"select r.cities, r.url from r" }
options = {}
options['enableCrossPartitionQuery'] = True
options['maxItemCount'] = 1000
q = client.QueryDocuments(collection['_self'], db_query, options)
results = list(q)

In [11]:
cities = []
for x in results:
    cities  = cities + x['cities']

In [12]:
def shorten_name(citylist):
    short = []
    for x in citylist:
        k = x.find(" lat")
        short.append(x[:k])
    return short

In [13]:
cities = shorten_name(cities)

In [14]:
x = set(cities)
uniquecities = list(x)
len(uniquecities)

2213

cities appear in more than one story.   we are interested in stories of regional interest so let's find cities that were in at least 3 stores but no more than five.   we will use these for creating edges.   

start by building a histogram of cities counting the number of stories

In [15]:
histo = {}
for x in uniquecities:
    histo[x] = 0

In [16]:
for y in cities:
    histo[y] = histo[y]+1

In [17]:
linkcities = []
for x in uniquecities:
    if histo[x]> 2 and histo[x] < 6:
        linkcities.append(x)

In [18]:
linkcities

['Tibet, Xizang, China',
 'Chieh Chung, Henan, China',
 'Havana, Ciudad de La Habana, Cuba',
 'Juba, Wilayat Bahr al Jabal, South Sudan',
 "Odessa, Odes'ka Oblast, Ukraine",
 'The White House, District of Columbia, United States',
 'Tampa Bay, Florida, United States',
 'Anambra, Nigeria (general), Nigeria',
 'Niger Delta, Nigeria (general), Nigeria',
 'Tokyo Bay, Japan (general), Japan',
 'Irvine, California, United States',
 'Batumi, Ajaria**, Georgia',
 'Khan Shaykhun, Idlib, Syria',
 'Jeddah, Makkah, Saudi Arabia',
 'Miramar National Cemetery, California, United States',
 'Byron Bay, New South Wales, Australia',
 'Karabakh, Dagestan, Russia',
 'Dutch Bank, United Kingdom (general), United Kingdom',
 'Tawang, Shaanxi, China',
 'Bridgetown, Saint Michael, Barbados',
 'Zhejiang, Zhejiang, China',
 'Jefferson City, Missouri, United States',
 'Latvia',
 'North Point, Hong Kong (general), Hong Kong',
 'Burkina Faso',
 'Chosin, Hamgyong-namdo, North Korea',
 'Kashmiri, Punjab, Pakistan',
 

In [19]:
len(linkcities)

245

In [20]:
len(results)

1699

In [21]:
#create a list of sets
#for each city in linkcities:
#    stories[city]= set of stories linked by city
#    
stories = {}
for city in linkcities:
    stories[city] = []
linkcitiesset = set(linkcities)
#print(linkcitiesset)
for res in results:
    rescit = shorten_name(res['cities'])
    for c in rescit:
        if c in linkcitiesset:
            stories[c].append(res['url'])
#remove strange duplicate stories
for city in linkcities:
    x = stories[city]
    xset = set(x)
    stories[city] = list(xset)


In [22]:
stories

{'Accra, Greater Accra, Ghana': ['http://cajnewsafrica.com/2017/06/30/varsity-graduate-salvages-west-africas-youth-from-terrorism/',
  'http://www.pambazuka.org/pan-africanism/pan-african-agenda-21st-century'],
 'Akinci, Siirt, Turkey': ['https://www.pressenza.com/2017/06/un-hopes-success-cyprus-talks-enter-new-phase/',
  'http://famagusta-gazette.com/turkey-denies-claims-it-will-pull-troops-from-cyprus-p41073-69.htm'],
 'Aklan, Quirino, Philippines': ['http://news.pia.gov.ph/article/view/1541498786066/prrd-consoles-mother-of-cop-slain-by-npa',
  'http://news.pia.gov.ph/article/view/3441498802882/duterte-avows-comfortable-life-for-all',
  'http://www.philstar.com/region/2017/07/01/1715272/negros-occidental-electric-coop-ppc-inks-power-purchase-deal'],
 'Al-Bab, ?alab, Syria': ['https://www.dailysabah.com/syrian-crisis/2017/06/30/daesh-fully-withdraws-from-syrias-aleppo-province-monitor-says',
  'http://www.middle-east-online.com/english/?id=83719'],
 'Albania': ['http://www.staradverti

## build the graph
we will use the linkcities for the nodes and we link two cities if they share a common story.

In [23]:
edges = []
for i in range(0, len(linkcities)):
    for j in range(0, len(linkcities)):
        if i < j:
            iset = set(stories[linkcities[i]])
            jset = set(stories[linkcities[j]])
            kset = iset.intersection(jset)
            if len(kset)> 0:
                edges.append((i,j))

In [24]:
len(edges)

210

In [25]:
#!pip install python-igraph

In [26]:
import igraph as ig

In [27]:
G=ig.Graph(edges, directed=False)

In [28]:
#this is a not-very-good attempt to extract the name of the story from the url
def trimstory(s):
    k = s.find("-")
    if k > 1:
        v = s[:k].rfind('/')
        x = s[v+1:]
    else:
        x = s
    k = x.rfind("/")
    if k > 7:
        x = x[:k]
    k = x.rfind("?")
    if k > -1:
        x = x[:k]
    if len(x) < 6:
        return s
    return x

In [29]:
labels=[]
group=[]
for city in linkcities:
    st = ''
    for s in stories[city]:
        s = trimstory(s)
        if len(s) > 10:
            st = st + "<br>" + s
    labels.append(city+ "<br>"+st)
    group.append(len(stories[city]))

In [30]:
layt=G.layout('kk', dim=3)

In [31]:
layt[5]

[0.28300559161232136, 5.815968779972684, 7.491703992923659]

In [32]:
N = len(linkcities)
len(layt)

245

In [33]:
len(layt)

245

In [34]:
N = len(layt)

In [35]:
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]


In [36]:
#!pip install plotly

## use Plotly to make the plot
the text below is almost a direct copy of the plotly graph example.   the nodes will be distributed near the surface of a sphere in 3D.   

In [37]:
import plotly.plotly as py
from plotly.graph_objs import *
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [38]:
print(__version__)

2.0.12


In [39]:
init_notebook_mode(connected=True)

In [40]:
trace1=Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=Line(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=Marker(symbol='dot',
                             size=6,
                             color=group,
                             colorscale='Viridis',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

In [41]:
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )


In [42]:
layout = Layout(
         title="Network of shared news items. Nodes are cities. <br> They are linked if they each publised the same top story  ",
         width=1000,
         height=1000,
         showlegend=False,
         scene=Scene(
         xaxis=XAxis(axis),
         yaxis=YAxis(axis),
         zaxis=ZAxis(axis),
         ),
     margin=Margin(
        t=100
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False,
            text="",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=Font(
            size=14
            )
            )
        ]),    )

In [43]:
py.plotly.tools.set_credentials_file(username='dbgannon', api_key='losja5d9ws')

In [44]:
data=Data([trace1, trace2])
fig=Figure(data=data, layout=layout)

py.iplot(fig,  filename='urls')

INFO:Starting new HTTPS connection (1): plot.ly
